<img align="left" src="https://lever-client-logos.s3.amazonaws.com/864372b1-534c-480e-acd5-9711f850815c-1524247202159.png" width=200>
<br></br>
<br></br>

## *Data Science Unit 4 Sprint 3 Assignment 1*

# Recurrent Neural Networks and Long Short Term Memory (LSTM)

![Monkey at a typewriter](https://upload.wikimedia.org/wikipedia/commons/thumb/3/3c/Chimpanzee_seated_at_typewriter.jpg/603px-Chimpanzee_seated_at_typewriter.jpg)

It is said that [infinite monkeys typing for an infinite amount of time](https://en.wikipedia.org/wiki/Infinite_monkey_theorem) will eventually type, among other things, the complete works of Wiliam Shakespeare. Let's see if we can get there a bit faster, with the power of Recurrent Neural Networks and LSTM.

This text file contains the complete works of Shakespeare: https://www.gutenberg.org/files/100/100-0.txt

Use it as training data for an RNN - you can keep it simple and train character level, and that is suggested as an initial approach.

Then, use that trained RNN to generate Shakespearean-ish text. Your goal - a function that can take, as an argument, the size of text (e.g. number of characters or lines) to generate, and returns generated text of that size.

Note - Shakespeare wrote an awful lot. It's OK, especially initially, to sample/use smaller data and parameters, so you can have a tighter feedback loop when you're trying to get things running. Then, once you've got a proof of concept - start pushing it more!

In [22]:
"""
Import Statments:
"""
import numpy as np
import requests 

from tensorflow.keras.callbacks import LambdaCallback
from tensorflow.keras.preprocessing import sequence
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Embedding
from tensorflow.keras.layers import Dense, LSTM
from tensorflow.keras.optimizers import RMSprop

import random
import sys
import os

In [7]:
url = "https://www.gutenberg.org/files/100/100-0.txt"
r = requests.get(url)
r.encoding = r.apparent_encoding
data = r.text

len(data)

5740053

In [37]:
data[-5000:]

'tm\r\nwork, (b) alteration, modification, or additions or deletions to any\r\nProject Gutenberg-tm work, and (c) any Defect you cause.\r\n\r\nSection 2. Information about the Mission of Project Gutenberg-tm\r\n\r\nProject Gutenberg-tm is synonymous with the free distribution of\r\nelectronic works in formats readable by the widest variety of computers\r\nincluding obsolete, old, middle-aged and new computers. It exists\r\nbecause of the efforts of hundreds of volunteers and donations from\r\npeople in all walks of life.\r\n\r\nVolunteers and financial support to provide volunteers with the\r\nassistance they need are critical to reaching Project Gutenberg-tm’s\r\ngoals and ensuring that the Project Gutenberg-tm collection will remain\r\nfreely available for generations to come. In 2001, the Project Gutenberg\r\nLiterary Archive Foundation was created to provide a secure and\r\npermanent future for Project Gutenberg-tm and future generations. To\r\nlearn more about the Project Gutenber

In [38]:
# Encode Data as Chars

# Gather all text 
# Why? 1. See all possible characters 2. For training / splitting later
text = " ".join(data)

# Unique Characters
chars = list(set(text))

# Lookup Tables
char_int = {c:i for i, c in enumerate(chars)} 
int_char = {i:c for i, c in enumerate(chars)} 

In [39]:
len(chars)

107

In [40]:
# Create the sequence data

maxlen = 40
step = 5

encoded = [char_int[c] for c in text]

sequences = [] # Each element is 40 chars long
next_char = [] # One element for each sequence

for i in range(0, len(encoded) - maxlen, step):
    
    sequences.append(encoded[i : i + maxlen])
    next_char.append(encoded[i + maxlen])
    
print('sequences: ', len(sequences))

sequences:  2296013


In [41]:
sequences[0]

[60,
 15,
 19,
 15,
 81,
 15,
 46,
 15,
 84,
 15,
 70,
 15,
 39,
 15,
 64,
 15,
 51,
 15,
 15,
 15,
 35,
 15,
 97,
 15,
 51,
 15,
 39,
 15,
 102,
 15,
 71,
 15,
 39,
 15,
 46,
 15,
 31,
 15,
 65,
 15]

In [14]:
max_features = 20000
# cut texts after this number of words (among top max_features most common words)
maxlen = 80
batch_size = 32

In [17]:
model = Sequential()

model.add(Embedding(max_features, 128))
model.add(LSTM(128, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(1, activation='sigmoid'))

model.compile(loss='binary_crossentropy',
              optimizer='adam', 
              metrics=['accuracy'])

model.summary()

Model: "sequential_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, None, 128)         2560000   
_________________________________________________________________
lstm_1 (LSTM)                (None, 128)               131584    
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 129       
Total params: 2,691,713
Trainable params: 2,691,713
Non-trainable params: 0
_________________________________________________________________


In [24]:
# Encode Data as Chars

# Gather all text 
# Why? 1. See all possible characters 2. For training / splitting later
text = " ".join(data)

# Unique Characters
chars = list(set(text))

# Lookup Tables
char_int = {c:i for i, c in enumerate(chars)} 
int_char = {i:c for i, c in enumerate(chars)} 

In [25]:
len(chars)

107

In [28]:
# Create x & y

x = np.zeros((len(sequences), maxlen, len(chars)), dtype=np.bool)
y = np.zeros((len(sequences),len(chars)), dtype=np.bool)

for i, sequence in enumerate(sequences):
    for t, char in enumerate(sequence):
        x[i,t,char] = 1
        
    y[i, next_char[i]] = 1

In [29]:
x.shape, y.shape

((2296013, 40, 107), (2296013, 107))

In [31]:
%%time

# build the model: a single LSTM
model = Sequential()
model.add(LSTM(128, input_shape=(maxlen, len(chars))))
model.add(Dense(len(chars), activation='softmax'))

model.compile(loss='categorical_crossentropy', optimizer='adam')

def sample(preds):
    # helper function to sample an index from a probability array
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds) / 1
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    return np.argmax(probas)

def on_epoch_end(epoch, _):
    # Function invoked at end of each epoch. Prints generated text.
    
    print()
    print('----- Generating text after Epoch: %d' % epoch)
    
    start_index = random.randint(0, len(text) - maxlen - 1)
    
    generated = ''
    
    sentence = text[start_index: start_index + maxlen]
    generated += sentence
    
    print('----- Generating with seed: "' + sentence + '"')
    sys.stdout.write(generated)
    
    for i in range(400):
        x_pred = np.zeros((1, maxlen, len(chars)))
        for t, char in enumerate(sentence):
            x_pred[0, t, char_int[char]] = 1
            
        preds = model.predict(x_pred, verbose=0)[0]
        next_index = sample(preds)
        next_char = int_char[next_index]
        
        sentence = sentence[1:] + next_char
        
        sys.stdout.write(next_char)
        sys.stdout.flush()
    print()


print_callback = LambdaCallback(on_epoch_end=on_epoch_end)

# fit the model

model.fit(x, y,
          batch_size=32,
          epochs=2,
          callbacks=[print_callback])

Train on 2296013 samples
Epoch 1/2
2295968/2296013 [============================>.] - ETA: 0s - loss: 0.9590
----- Generating text after Epoch: 0
----- Generating with seed: "g   A d o n i s   s i t t i n g   b y   "
    A d o n i s   s i t t i n g   b y   T r e e n g ? 
 S C E N V A L 
     E n t e r   S T E R D ,   I 
 _ l o o t i s ,   a s   h i s   s u d   f r o m   t h e   r i g h t   o f   m y 
 T o   w e l l w e r y   e n t r e n o w ,   y o u r   p u c h   a n   y o u s   w a l l   o f   d o ’ t s , 
         A n d s t   t h e m . 
     L U C E N T O . 
 I   b e a r d   y o u r   a r e   s u c h   l i v i l l ,   I ,   w a n k 
2296013/2296013 [==============================] - 2769s 1ms/sample - loss: 0.9590
Epoch 2/2
2296000/2296013 [============================>.] - ETA: 0s - loss: 0.8113
----- Generating text after Epoch: 1
----- Generating with seed: " s h e   l o v ’ d   p r o v ’ d   m a d"
 s h e   l o v ’ d   p r o v ’ d   m a d e ? 
     F U L S .   '

/Users/jorge/opt/anaconda3/envs/U4-S1-NLP-DS10/lib/python3.7/site-packages/ipykernel_launcher.py:11: RuntimeWarning: divide by zero encountered in log
  # This is added back by InteractiveShellApp.init_path()


 T w i t i n g   I   j e c k e n e s s   t h e   d o o r ,   y i n k e d   i n t e r t e n c e   b e   b e g   o l d   d i r c h u r e n t   a r e   o n g e n ,   w i t h   m i g h t , 
 A l a s l y   p o w e r   s t e r s a b e r   o f   e v e r   l i k e   o w r y ;   M a j e f t   p r o v e ;   a n d   h a v e   a l p o m e   t h a t   v a r e .   W h i c h   t h e r
2296013/2296013 [==============================] - 2665s 1ms/sample - loss: 0.8113
CPU times: user 4h 4min 8s, sys: 1h 8min 49s, total: 5h 12min 58s
Wall time: 1h 30min 34s


# Resources and Stretch Goals

## Stretch goals:
- Refine the training and generation of text to be able to ask for different genres/styles of Shakespearean text (e.g. plays versus sonnets)
- Train a classification model that takes text and returns which work of Shakespeare it is most likely to be from
- Make it more performant! Many possible routes here - lean on Keras, optimize the code, and/or use more resources (AWS, etc.)
- Revisit the news example from class, and improve it - use categories or tags to refine the model/generation, or train a news classifier
- Run on bigger, better data

## Resources:
- [The Unreasonable Effectiveness of Recurrent Neural Networks](https://karpathy.github.io/2015/05/21/rnn-effectiveness/) - a seminal writeup demonstrating a simple but effective character-level NLP RNN
- [Simple NumPy implementation of RNN](https://github.com/JY-Yoon/RNN-Implementation-using-NumPy/blob/master/RNN%20Implementation%20using%20NumPy.ipynb) - Python 3 version of the code from "Unreasonable Effectiveness"
- [TensorFlow RNN Tutorial](https://github.com/tensorflow/models/tree/master/tutorials/rnn) - code for training a RNN on the Penn Tree Bank language dataset
- [4 part tutorial on RNN](http://www.wildml.com/2015/09/recurrent-neural-networks-tutorial-part-1-introduction-to-rnns/) - relates RNN to the vanishing gradient problem, and provides example implementation
- [RNN training tips and tricks](https://github.com/karpathy/char-rnn#tips-and-tricks) - some rules of thumb for parameterizing and training your RNN